In [1]:
import os
import torch
import numpy as np
import torchvision
from PIL import Image
import matplotlib.pyplot as plt

import torchvision.transforms as T
import torch.nn.functional as F

from ig_pkg.datasets import get_datasets

from ig_pkg.models.generator import get_model
from ig_pkg.models.classifier import get_classifier
from ig_pkg.models.pretrained_models import get_pretrained_model

from ig_pkg.inputattribs.ig import ig
from ig_pkg.inputattribs.baseline_generator import get_baseline_generator

from ig_pkg.misc import process_heatmap, normalize_tensor, convert_to_img, label_to_class, tran, na_imshow

IMAGENET_MEAN = [0.485, 0.456, 0.406]
IMAGENET_STD  = [0.229, 0.224, 0.225]


In [24]:
# data_path="/root/data/ILSVRC2012_val/"

# _, valid_datasets = get_datasets("imagenet1k", data_path)
# _, valid_datasets2 = get_datasets("imagenet1k", data_path, transform=T.Compose([T.Resize(256),T.CenterCrop(256),]) )

device="cuda:0"
# model = get_pretrained_model("vgg16").to(device)

# train_dataset, valid_dataset = get_datasets(name= 'celebAHQ_identity', data_path = '/root/data/CelebA_HQ_facial_identity_dataset')
# class_names = train_dataset.classes
# np.save('/root/data/celebAHQ_identity_class_names.npy', class_names)
# class_names = np.load('/root/data/celebAHQ_identity_class_names.npy')

# train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=2)
# test_dataloader = torch.utils.data.DataLoader(valid_dataset, batch_size=16, shuffle=True, num_workers=2)

# image_path = '/root/data/CelebA_HQ_facial_identity_dataset/train'

In [ ]:
import time
import torch.nn as nn
import torch.optim as optim
from torch.optim.lr_scheduler import StepLR
from ig_pkg.loss.focal_loss import FocalLoss
from ig_pkg.loss.metrics import ArcMarginProduct, AddMarginProduct
import torchvision.models as models

device="cuda:0"
train_dataset, valid_dataset = get_datasets(name= 'celebAHQ_whole', data_path = '/root/data/whole') # 3819,2398 
train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size=64, shuffle=True, num_workers=2)

model = models.resnet18(pretrained=True)
num_features = model.fc.in_features # 512
model.fc = nn.Linear(num_features, 3819) # multi-class classification (num_of_class == 307)
model = model.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)

num_epochs = 30
start_time = time.time()

for epoch in range(num_epochs):
    """ Training Phase """
    model.train()

    running_loss = 0.
    running_corrects = 0

    # load a batch data of images
    for i, (inputs, labels) in enumerate(train_dataloader):
        inputs = inputs.to(device)
        labels = labels.to(device)

        # forward inputs and get output
        optimizer.zero_grad()
        outputs = model(inputs)
        _, preds = torch.max(outputs, 1)
        loss = criterion(outputs, labels)

        # get loss value and update the network weights
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * inputs.size(0)
        running_corrects += torch.sum(preds == labels.data)

    epoch_loss = running_loss / len(train_dataset)
    epoch_acc = running_corrects / len(train_dataset) * 100.
    print('[Train #{}] Loss: {:.4f} Acc: {:.4f}% Time: {:.4f}s'.format(epoch, epoch_loss, epoch_acc, time.time() - start_time))

save_path = f'/root/pretrained/celebahq_whole_{epoch_acc}.pth'
torch.save(model.state_dict(), save_path)

In [12]:
import time
import torch.nn as nn
import torch.optim as optim
from torch.optim.lr_scheduler import StepLR
from ig_pkg.loss.focal_loss import FocalLoss
from ig_pkg.loss.metrics import ArcMarginProduct, AddMarginProduct
import torchvision.models as models

device="cuda:0"
train_dataset, valid_dataset = get_datasets(name= 'celebAHQ_5', data_path = '/root/data/train') # 3819,2398 
train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size=64, shuffle=True, num_workers=2)

model = models.resnet18(pretrained=True)
num_features = model.fc.in_features # 512
model.fc = nn.Linear(num_features, 2398) # multi-class classification (num_of_class == 307)
model = model.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)

num_epochs = 30
start_time = time.time()

for epoch in range(num_epochs):
    """ Training Phase """
    model.train()

    running_loss = 0.
    running_corrects = 0

    # load a batch data of images
    for i, (inputs, labels) in enumerate(train_dataloader):
        inputs = inputs.to(device)
        labels = labels.to(device)

        # forward inputs and get output
        optimizer.zero_grad()
        outputs = model(inputs)
        _, preds = torch.max(outputs, 1)
        loss = criterion(outputs, labels)

        # get loss value and update the network weights
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * inputs.size(0)
        running_corrects += torch.sum(preds == labels.data)

    epoch_loss = running_loss / len(train_dataset)
    epoch_acc = running_corrects / len(train_dataset) * 100.
    print('[Train #{}] Loss: {:.4f} Acc: {:.4f}% Time: {:.4f}s'.format(epoch, epoch_loss, epoch_acc, time.time() - start_time))
    if epoch_acc > 99.9: break
        
save_path = f'/root/pretrained/celebahq_train_{epoch_acc}.pth'
torch.save(model.state_dict(), save_path)

[Train #0] Loss: 7.2841 Acc: 1.9056% Time: 103.2269s
[Train #1] Loss: 5.3779 Acc: 14.7313% Time: 200.3553s
[Train #2] Loss: 3.7013 Acc: 35.2106% Time: 296.9381s
[Train #3] Loss: 2.3991 Acc: 57.8945% Time: 394.6895s
[Train #4] Loss: 1.4377 Acc: 77.4545% Time: 491.7799s
[Train #5] Loss: 0.7544 Acc: 91.3602% Time: 589.7661s
[Train #6] Loss: 0.3378 Acc: 97.9025% Time: 687.1586s
[Train #7] Loss: 0.1383 Acc: 99.6341% Time: 783.9427s
[Train #8] Loss: 0.0699 Acc: 99.8393% Time: 880.5901s
[Train #9] Loss: 0.0476 Acc: 99.8661% Time: 979.0721s
[Train #10] Loss: 0.0416 Acc: 99.8304% Time: 1076.6075s
[Train #11] Loss: 0.0305 Acc: 99.8974% Time: 1173.7527s
[Train #12] Loss: 0.0257 Acc: 99.9018% Time: 1271.6747s


In [2]:
import time
import torch.nn as nn
import torch.optim as optim
from torch.optim.lr_scheduler import StepLR
from ig_pkg.loss.focal_loss import FocalLoss
from ig_pkg.loss.metrics import ArcMarginProduct, AddMarginProduct
import torchvision.models as models

device="cuda:0"
train_dataset, valid_dataset = get_datasets(name= 'celebAHQ_5', data_path = '/root/data/identity_celebahq') # 3819,2398 
train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size=64, shuffle=True, num_workers=2)

class_names = train_dataset.classes
# np.save('/root/data/celebahq_identity/identity_original.npy', class_names)

model = models.resnet18(pretrained=True)
num_features = model.fc.in_features # 512
model.fc = nn.Linear(num_features, 6217) 
model = model.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)

num_epochs = 30
start_time = time.time()

for epoch in range(num_epochs):
    """ Training Phase """
    model.train()

    running_loss = 0.
    running_corrects = 0

    # load a batch data of images
    for i, (inputs, labels) in enumerate(train_dataloader):
        inputs = inputs.to(device)
        labels = labels.to(device)

        # forward inputs and get output
        optimizer.zero_grad()
        outputs = model(inputs)
        _, preds = torch.max(outputs, 1)
        loss = criterion(outputs, labels)

        # get loss value and update the network weights
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * inputs.size(0)
        running_corrects += torch.sum(preds == labels.data)

    epoch_loss = running_loss / len(train_dataset)
    epoch_acc = running_corrects / len(train_dataset) * 100.
    print('[Train #{}] Loss: {:.4f} Acc: {:.4f}% Time: {:.4f}s'.format(epoch, epoch_loss, epoch_acc, time.time() - start_time))
    if epoch_acc > 99: break
        
save_path = f'/root/pretrained/celebahq_original_{epoch_acc}.pth'
torch.save(model.state_dict(), save_path)

/root/dh/lib/python3.8/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/root/dh/lib/python3.8/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


[Train #0] Loss: 8.1924 Acc: 1.0633% Time: 133.8641s
[Train #1] Loss: 6.5828 Acc: 8.9867% Time: 270.5878s
[Train #2] Loss: 5.1467 Acc: 23.5567% Time: 410.5533s
[Train #3] Loss: 3.9258 Acc: 39.5900% Time: 550.8081s
[Train #4] Loss: 2.9015 Acc: 55.1833% Time: 690.3970s
[Train #5] Loss: 2.0580 Acc: 68.9933% Time: 831.7129s
[Train #6] Loss: 1.3593 Acc: 81.0033% Time: 969.8034s
[Train #7] Loss: 0.8327 Acc: 89.6933% Time: 1108.4917s
[Train #8] Loss: 0.4637 Acc: 95.1233% Time: 1248.7256s
[Train #9] Loss: 0.2327 Acc: 98.8367% Time: 1383.8723s
[Train #10] Loss: 0.1083 Acc: 99.7600% Time: 1523.6631s


In [29]:
# model = get_classifier('resnet', 'data').to(device)

model.eval()

with torch.no_grad():
    running_loss = 0.
    running_corrects = 0
    i = 0
    for inputs, labels in test_dataloader:
        i += 1
        inputs = inputs.to(device)
        labels = labels.to(device)        
                    
        outputs = model(inputs)
        outputs = metric_fc(outputs, labels)
        _, preds = torch.max(outputs, 1)        
        loss = criterion(outputs, labels)
        
        running_loss += loss.item() * inputs.size(0)
        running_corrects += torch.sum(preds == labels.data)
        
    print(i)
    print(inputs.size())
    print(labels.size())
    print(preds.size())
    print(len(valid_dataset))
    epoch_loss = running_loss / len(valid_dataset)
    epoch_acc = running_corrects / len(valid_dataset) * 100.
    print('[Test #{}] Loss: {:.4f} Acc: {:.4f}% Time: {:.4f}s'.format(epoch, epoch_loss, epoch_acc, time.time() - start_time))

76
torch.Size([15, 3, 224, 224])
torch.Size([15])
torch.Size([15])
1215
[Test #29] Loss: 2.4216 Acc: 49.2181% Time: 1565.9302s


In [18]:
import time
import torch.nn as nn

model = get_classifier('resnet', 'data')
model = model.to("cuda:0")
device="cuda:0"
criterion = nn.CrossEntropyLoss()

model.eval()
start_time = time.time()

with torch.no_grad():
    running_loss = 0.
    running_corrects = 0

    for i, (inputs, labels) in enumerate(test_dataloader):
        inputs = inputs.to(device)
        labels = labels.to(device)

        outputs = model(inputs)
        _, preds = torch.max(outputs, 1)
        loss = criterion(outputs, labels)

        running_loss += loss.item() * inputs.size(0)
        running_corrects += torch.sum(preds == labels.data)

#         if i == 0:
#             print('[Prediction Result Examples]')
#             images = torchvision.utils.make_grid(inputs[:4])
#             na_imshow(images.cpu(), title=[class_names[x] for x in labels[:4]])
#             images = torchvision.utils.make_grid(inputs[4:8])
#             na_imshow(images.cpu(), title=[class_names[x] for x in labels[4:8]])

    epoch_loss = running_loss / len(valid_dataset)
    epoch_acc = running_corrects / len(valid_dataset) * 100.
    print('[Test] Loss: {:.4f} Acc: {:.4f}% Time: {:.4f}s'.format(epoch_loss, epoch_acc, time.time() - start_time))
     

[Test] Loss: 0.5880 Acc: 86.0082% Time: 5.7431s


In [ ]:
# gen = get_model('stylegan', '/root/pretrained/stylegan-celebahq-1024x1024.pt', 1024)
gen = get_model('stylegan', '/root/pretrained/stylegan-celebahq-1024x1024.pt', 1024)
gen = gen.to('cuda:0')

latent = torch.rand((3, 512)).to("cuda:0")
images = gen(latent)

classifier = get_classifier('resnet', 'data')
classifier = classifier.to("cuda:0")

transform = T.Compose([
                T.Resize(224),
                T.CenterCrop(224),
                tran,
                T.Normalize(IMAGENET_MEAN, IMAGENET_STD)
            ])
img = transform(images)

score = classifier(img)
label = torch.argmax(score, dim = 1)
pred = label_to_class(label, class_names)
print(pred)


In [ ]:
MEAN = [0.5, 0.5, 0.5]
STD = [0.5, 0.5, 0.5]

IMAGENET_MEAN = [0.485, 0.456, 0.406]
IMAGENET_STD  = [0.229, 0.224, 0.225]

CIFAR100_MEAN = [0.4914, 0.4822, 0.4465]
CIFAR100_STD  = [0.2023, 0.1994, 0.2010] 

CIFAR10_MEAN = [0.4914, 0.4822, 0.4465]
CIFAR10_STD  = [0.2023, 0.1994, 0.2010]

MNIST_MEAN = [0.1307]
MNIST_STD  = [0.3081] 

temp = img[0].detach().cpu()
def na_imshow(input, mean, std):
#     input = input.numpy()
#     print(input.shape)
    input = input.numpy().transpose((1, 2, 0))
#     mean = np.array([0.485, 0.456, 0.406])
#     std = np.array([0.229, 0.224, 0.225])
    input = std * input + mean
    input = np.clip(input, 0, 1)
#     input[:, :, 0] += 0.1
#     input = input.permute
    plt.figure()
    plt.imshow(input)
    plt.show()   
    
# na_imshow(temp, MEAN, STD)
na_imshow(temp, IMAGENET_MEAN, IMAGENET_STD)
# na_imshow(temp, CIFAR100_MEAN, CIFAR100_STD)
# na_imshow(temp, CIFAR10_MEAN, CIFAR10_STD)
# na_imshow(temp, MNIST_MEAN, MNIST_STD)

In [ ]:
fig, axes = plt.subplots(2, 3, figsize = (15, 4))
axes = axes.flat

for i in range(len(cls)):
    ax = next(axes)
    x = img[i]    
    imga = convert_to_img(x)
    ax.imshow(imga)
    tlt = cls[i]
    ax.set_title(tlt)
    
image_path = '/root/data/CelebA_HQ_facial_identity_dataset/train'


for i in range(len(cls)):
    img_directory = os.path.join(image_path, cls[i])    
    img_path_list = os.listdir(img_directory)
    img_path = os.path.join(img_directory, img_path_list[0])
#     print(img_path)
    im = Image.open(img_path)
    
    ax = next(axes)
    ax.imshow(im)    

plt.savefig('/root/ig_inversion/figure/baseline/1.png')